# Preprocess
This notebook is responsible for preprocessing the images. Run thi notebook if you want to train the models.

Contents:
- Crop (Remove background noise)
- Flip (Every car should be in the same direction)
- Scale (Scale all cars according to a reference car)
- Execute (Run the scripts)

What will you need to do?
- Create a dir named *data* in the root folder of the project
- Download the GP22 Dataset (https://zenodo.org/records/6366808), both Images.zip and Labels.zip
- Run code block below to generate directories

## 0. Configuration

### 0.1 Path

In [20]:
# configure paths here 
path_to_orientation_model = "../checkpoints/orientation_classifier_checkpoint.pth"
path_to_gp22_images = "../data/GP22/images"
path_to_gp22_labels = "../data/GP22/labels"
path_to_json_labels = "../data/json_labels"

# path_to_reference_car_label # Configured under 3rd code block in --> Execute
output_dir_images_cropped = "../data/processed/cropped"
output_dir_images_flipped = "../data/processed/flipped_images"
output_dir_labels_flipped = "../data/processed/flipped_labels"
output_dir_images_scaled = "../data/processed/scaled"

### 0.2 Import libraries

In [2]:
import os
import zipfile
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import torch
from torchvision import transforms, models
from PIL import Image
from pathlib import Path
import shutil
import json

### 0.3 Directories generation

In [3]:

def ensure_directories_exist(paths):
    """ 
    Ensure that the directories in the given paths exist. If they do not, create them.
    Args:
        paths (list): List of paths to directories to ensure exist.
    Returns:
        None
    """

    for path in paths:
        directory = path if path.endswith('/') or '.' not in os.path.basename(path) else os.path.dirname(path)
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"Created directory: {directory}")
        else:
            print(f"Directory already exists: {directory}")

In [4]:
# Configure paths here
paths = [
    "../data",
    "../data/models/orientation_model/best_model.pth",
    "../data/GP22/images/",
    "../data/GP22/labels/",
    "../data/json_labels/",
    "../data/processed/cropped/",
    "../data/processed/flipped_images/",
    "../data/processed/flipped_labels/",
    "../data/processed/scaled"
]


In [5]:
ensure_directories_exist(paths)

Directory already exists: ../data
Directory already exists: ../data/models/orientation_model
Directory already exists: ../data/GP22/images/
Directory already exists: ../data/GP22/labels/
Directory already exists: ../data/json_labels/
Directory already exists: ../data/processed/cropped/
Directory already exists: ../data/processed/flipped_images/
Directory already exists: ../data/processed/flipped_labels/
Directory already exists: ../data/processed/scaled


### 0.4 Unzip of GP22 dataset

Insert the downloaded Images.zip and Labels.zip into the data folder

In [6]:
# Define paths
base_dir = "../data/GP22"
images_zip = "../data/Images.zip"
labels_zip = "../data/Labels.zip"
images_dir = os.path.join(base_dir, "images")
labels_dir = os.path.join(base_dir, "labels")

# Function to remove __MACOSX folder if it exists
def remove_macosx_folder(base_dir):
    macosx_path = os.path.join(base_dir, "__MACOSX")
    if os.path.exists(macosx_path):
        shutil.rmtree(macosx_path)
        print(f"Removed {macosx_path}.")

# Check and unzip Images.zip
if os.path.exists(images_zip):
    if not os.listdir(images_dir):
        print(f"Extracting {images_zip} to {images_dir}...")
        with zipfile.ZipFile(images_zip, 'r') as zip_ref:
            zip_ref.extractall(base_dir) 
        print(f"Extraction complete: {images_dir}")
        os.remove(images_zip)
        print(f"Deleted {images_zip}.")
        remove_macosx_folder(base_dir)
    else:
        print(f"{images_dir} already contains files. Skipping extraction of {images_zip}.")
else:
    print(f"{images_zip} not found.")

# Check and unzip Labels.zip
if os.path.exists(labels_zip):
    if not os.listdir(labels_dir):
        print(f"Extracting {labels_zip} to {labels_dir}...")
        with zipfile.ZipFile(labels_zip, 'r') as zip_ref:
            zip_ref.extractall(base_dir)
        print(f"Extraction complete: {labels_dir}")
        os.remove(labels_zip)
        print(f"Deleted {labels_zip}.")
        remove_macosx_folder(base_dir)
    else:
        print(f"{labels_dir} already contains files. Skipping extraction of {labels_zip}.")
else:
    print(f"{labels_zip} not found.")


../data/Images.zip not found.
../data/Labels.zip not found.


## 1. Crop
Removing background noise slightly improves model performance.
The following code block will: 
- Use GP22 labels to process the flipped images and remove background

In [7]:
def crop_out_background(images_dir, labels_dir, output_dir):
    """
    Process all images in a directory and crop out the background using bounding boxes.

    Args:
        images_dir (str): Path to the directory containing the images.
        labels_dir (str): Path to the directory containing the labels.
        output_dir (str): Path to the directory to save the cropped images.
    Returns:
        None
    """
    os.makedirs(output_dir, exist_ok=True)

    for image_path in Path(images_dir).glob("*.jpg"):
        label_path = Path(labels_dir) / f"{image_path.stem}.txt"

        if not label_path.exists():
            print(f"No label found for {image_path.name}. Skipping.")
            continue

        image = cv2.imread(str(image_path))
        if image is None:
            print(f"Could not read the image at {image_path}. Skipping.")
            continue

        img_height, img_width = image.shape[:2]

        mask = np.zeros(image.shape[:2], dtype=np.uint8)

        bounding_boxes = []
        with open(label_path, 'r') as file:
            for line in file:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                bounding_boxes.append([x_center, y_center, width, height])

        for (x_center, y_center, width, height) in bounding_boxes:
            x = int((x_center - width / 2) * img_width)
            y = int((y_center - height / 2) * img_height)
            w = int(width * img_width)
            h = int(height * img_height)

            cv2.rectangle(mask, (x, y), (x + w, y + h), 255, thickness=-1)

        result = cv2.bitwise_and(image, image, mask=mask)

        output_path = Path(output_dir) / image_path.name
        cv2.imwrite(str(output_path), result)

## 2. Flip
Every car should be pointing in the same direction.
The following code blocks will:
- Load the orientation model
- Predict whether a car is pointing left or right
- Flip the corresponding label of each car
- Flip the image of the car itself

We have chosen to flip cars to the **left**. That is, nose points to the left.

### 2.1 Load the orientation model

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_orientation_model(checkpoint_path):
    """
    Loads the trained orientation model from the given checkpoint path.
    Args:
        checkpoint_path (str): Path to the checkpoint file.
    Returns:
        torch.nn.Module: The loaded model.
    """
    model = models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(model.fc.in_features, 2)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)
    model.eval()
    print(f"Model loaded from {checkpoint_path}")
    return model

### 2.2 Predict orientation

In [9]:
def predict_orientation(model, image_path):
    """
    Predicts the orientation of the car in the image.
    Args:
        model (torch.nn.Module): The orientation model.
        image_path (str): Path to the image.
    Returns:
        str: The predicted orientation ("left" or "right").
    """
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        return "left" if predicted.item() == 0 else "right"

### Flip x-coordinates of labels

In [10]:
def flip_labels_x(label_path, output_label_dir):
    """
    Flips the x-coordinates of labels and saves them to a new directory.

    Args:
        label_path (str): Path to the label file.
        output_label_dir (str): Path to the directory to save the flipped labels.
    Returns:
        None
    """
    os.makedirs(output_label_dir, exist_ok=True)

    flipped_labels = []
    with open(label_path, 'r') as f:
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            x_center = 1 - x_center  # Flip the x-coordinate
            flipped_labels.append(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    output_label_path = os.path.join(output_label_dir, os.path.basename(label_path))
    with open(output_label_path, 'w') as f:
        f.write("\n".join(flipped_labels))
    
    print(f"Flipped labels saved to {output_label_path}")

### 2.3 Flip images

In [11]:
def flip_image(image_path, output_image_dir):
    """
    Flips an image horizontally and saves it to the output directory.

    Args:
        image_path (str): Path to the image.
        output_image_dir (str): Path to the directory to save the flipped image.
    Returns:
        str: Path to the saved flipped image.
    """
    os.makedirs(output_image_dir, exist_ok=True)

    image = cv2.imread(str(image_path))
    flipped_image = cv2.flip(image, 1)
    output_image_path = os.path.join(output_image_dir, os.path.basename(image_path))
    cv2.imwrite(output_image_path, flipped_image)
    
    print(f"Flipped image saved to {output_image_path}")
    return output_image_path

### 2.4 Flip Images and Labels
Run this to process all images and labels

In [12]:
def flip_images_and_labels(model, images_dir, labels_dir, output_image_dir, output_label_dir):
    """
    Processes all images: identifies flipped images, flips them, and flips the labels' x-coordinates.

    Args:
        model (torch.nn.Module): The orientation model.
        images_dir (str): Path to the directory containing the images.
        labels_dir (str): Path to the directory containing the labels.
        output_image_dir (str): Path to the directory to save the flipped images.
        output_label_dir (str): Path to the directory to save the flipped labels.
    Returns:
        None
    """
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    for image_path in Path(images_dir).glob("*.jpg"):
        label_path = Path(labels_dir) / f"{image_path.stem}.txt"
        
        if label_path.exists():
            orientation = predict_orientation(model, str(image_path))
            flipped = False

            if orientation == "right":
                flipped = True
                flip_image(str(image_path), output_image_dir)
                flip_labels_x(str(label_path), output_label_dir)
            else:
                shutil.copy(str(image_path), os.path.join(output_image_dir, image_path.name))
                shutil.copy(str(label_path), os.path.join(output_label_dir, label_path.name))
                print(f"Image and labels copied to {output_image_dir} and {output_label_dir}")
        else:
            print(f"Label file not found for {image_path}")


## 3. Scale
Cars should be relative to each other in size.
The following code blocks will:
- Calculate rim area (used as a reference when scaling)
- Calculate scaling factor (based on reference car and current car)
- Scale images and update labels

### 3.1 Rim area

In [13]:
def calculate_rim_area_of_front_wheel(label_path, resolution=1024):
    """
    Returns {area} of front-wheel rim
    Important! Car nose should be pointing left

    Args:
        label_path (str): Path to the label file.
        resolution (int): Resolution of the image.
    Returns:
        float: Area of the front-wheel rim in the image.
    """
    smallest_x = float("inf")
    smallest_box = None

    with open(label_path, "r") as file:
        for line in file:
            class_id, x_center, y_center, width, height = line.strip().split()
            if int(class_id) == 1: 
                x_center = float(x_center)
                if x_center < smallest_x:
                    smallest_x = x_center
                    smallest_box = (float(width), float(height))

    if smallest_box:
        width, height = smallest_box
        area = width * height * (resolution**2)
        return area
    else:
        return 0

### 3.2 Scaling factor

In [14]:
def compare_area(rim_area_reference_car, rim_area_current_car):
    """
    Calculate the scaling factor to adjust the dimensions of the current car
    so that its area matches the area of the reference car.

    Args:
        rim_area_reference_car (float): Area of the front-wheel rim of the reference car.
        rim_area_current_car (float): Area of the front-wheel rim of the current car.
    Returns:
        float: Scaling factor to adjust the dimensions of the current car.
    """
    if rim_area_reference_car <= 0 or rim_area_current_car <= 0:
        raise ValueError("Both areas must be positive numbers.")
    
    scaling_factor = (rim_area_reference_car / rim_area_current_car) ** 0.5
    
    return scaling_factor

### 3.3 Scale JSON labels

In [15]:
def update_json_labels(json_path, scaling_factor, offset_x, offset_y, labels_dir):
    """
    Updates the JSON labels to match the scaled and padded image.

    Args:
        json_path (str): Path to the JSON label file.
        scaling_factor (float): Scaling factor to adjust the dimensions of the current car.
        offset_x (int): The x-offset to center the points.
        offset_y (int): The y-offset to center the points.
        labels_dir (str): Path to the directory to save the updated JSON labels.
    Returns:
        None
    """
    with open(json_path, 'r') as file:
        data = json.load(file)

    # Scale and offset points
    for shape in data['shapes']:
        new_points = []
        for point in shape['points']:
            # First scale the points
            scaled_x = point[0] * scaling_factor
            scaled_y = point[1] * scaling_factor
            
            # Then add the positive offset to center the points
            scaled_x += offset_x  # Remove the negative sign
            scaled_y += offset_y  # Remove the negative sign

            # Only include points that fall within the 1024x1024 canvas
            if 0 <= scaled_x < 1024 and 0 <= scaled_y < 1024:
                new_points.append([scaled_x, scaled_y])

        shape['points'] = new_points

    # Update image metadata
    data['imageWidth'] = 1024
    data['imageHeight'] = 1024

    # Save updated JSON
    json_filename = os.path.basename(json_path)
    updated_json_path = os.path.join(labels_dir, json_filename)

    with open(updated_json_path, 'w') as file:
        json.dump(data, file, indent=4)

### 3.4 Scale images


In [16]:
def scale_and_crop_image(image_path, scaling_factor, label_path, json_label_path, output_dir):
    """
    Scales an image by a given scaling factor, ensures it is padded to 1024x1024 pixels,
    and scales corresponding JSON labels.

    Args:
        image_path (str): Path to the image.
        scaling_factor (float): Scaling factor to adjust the dimensions of the current car.
        label_path (str): Path to the label file.
        json_label_path (str): Path to the JSON label file.
        output_dir (str): Path to the directory to save the scaled image.
    Returns:
        None
    """
    if scaling_factor <= 0:
        raise ValueError("Scaling factor must be a positive number.")

    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image file not found: {image_path}")

    # Get original dimensions
    original_height, original_width = image.shape[:2]

    # Compute new dimensions
    new_width = int(original_width * scaling_factor)
    new_height = int(original_height * scaling_factor)

    # Resize the image using OpenCV
    scaled_image = cv2.resize(
        image, (new_width, new_height), interpolation=cv2.INTER_LINEAR
    )

    # Ensure the image is 1024x1024 by padding with black if necessary
    target_size = 1024
    padded_image = np.zeros((target_size, target_size, 3), dtype=np.uint8)

    # Center the scaled image in the 1024x1024 canvas
    offset_x = (target_size - new_width) // 2 if new_width < target_size else 0
    offset_y = (target_size - new_height) // 2 if new_height < target_size else 0

    insert_width = min(new_width, target_size)
    insert_height = min(new_height, target_size)

    padded_image[offset_y:offset_y+insert_height, offset_x:offset_x+insert_width] = \
        scaled_image[:insert_height, :insert_width]

    # Create directories for images and labels
    images_dir = os.path.join(output_dir, "images")
    labels_dir = os.path.join(output_dir, "labels")
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    # Save padded image
    image_filename = os.path.basename(image_path)
    padded_image_path = os.path.join(images_dir, image_filename)
    cv2.imwrite(padded_image_path, padded_image)

    # Update JSON labels
    update_json_labels(json_label_path, scaling_factor, offset_x, offset_y, labels_dir)

## 4. Execute
The following code blocks will execute corresponding code for:
- Removing background
- Flipping images and labels
- Scaling images and labels

### 4.1 Crop

In [17]:
path_to_gp22_images          # Directory with GP22 images
path_to_gp22_labels          # Directory with bounding box labels (text files)
output_dir_images_cropped    # Directory to save cropped images

crop_out_background(path_to_gp22_images, path_to_gp22_labels, output_dir_images_cropped)

### 4.2 Flip

In [22]:
path_to_orientation_model   # Directory with orientation model
output_dir_images_cropped   # Directory with cropped images
path_to_json_labels         # Directory with JSON labels
output_dir_images_flipped   # Directory to save flipped images
output_dir_labels_flipped   # Directory to save flipped labels

model = load_orientation_model(path_to_orientation_model)

flip_images_and_labels(
    model, 
    output_dir_images_cropped, 
    path_to_json_labels, 
    output_dir_images_flipped, 
    output_dir_labels_flipped
)

/Users/mattiacarlino/Politecnico coding/Safety-System-Design/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/mattiacarlino/Politecnico coding/Safety-System-Design/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model loaded from ../checkpoints/orientation_classifier_checkpoint.pth
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, aff

/var/folders/w7/n9y0bc4n5zgb8gg118j_j66h0000gn/T/ipykernel_2149/3320539876.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_

### 4.3 Scale

In [ ]:
def scale_all_images_and_labels(reference_label_path, images_folder, labels_folder, json_labels_folder, output_folder, resolution=1024):
    """
    Scales images and corresponding labels
    """
    # Create output directories
    images_output_dir = os.path.join(output_folder, "images")
    labels_output_dir = os.path.join(output_folder, "labels")
    os.makedirs(images_output_dir, exist_ok=True)
    os.makedirs(labels_output_dir, exist_ok=True)

    # Ensure the reference label path is valid
    if not os.path.exists(reference_label_path):
        print(f"Reference label path {reference_label_path} does not exist.")
        return
    
    reference_area = calculate_rim_area_of_front_wheel(reference_label_path, resolution)
    print(f"Reference area: {reference_area}")
    
    # Loop through label files in the label folder
    print(f"Looking for label files in: {labels_folder}")
    for label_file in os.listdir(labels_folder):
        if label_file.endswith(".txt"):
            print(f"Found label file: {label_file}")
            label_path = os.path.join(labels_folder, label_file)
            
            current_area = calculate_rim_area_of_front_wheel(label_path, resolution)
            print(f"Current area for {label_file}: {current_area}")
            
            # Get corresponding image file
            image_name = label_file.replace(".txt", "_aug_0.jpg")  # Assuming images are .jpg
            image_path = Path(images_folder) / image_name  # Use Path to handle path joining
            image_path = str(image_path).replace("\\", "/")
            print(f"Looking for image: {image_path}")
            
            if not os.path.exists(image_path):
                print(f"Image {image_name} corresponding to label {label_file} does not exist. Skipping.")
                continue

            # Get corresponding JSON label file (assumes the JSON label has the same name as the .txt label)
            json_label_name = label_file.replace(".txt", ".json")
            json_label_path = os.path.join(json_labels_folder, json_label_name)
            print(f"Looking for JSON label: {json_label_path}")
            
            if not os.path.exists(json_label_path):
                print(f"JSON label {json_label_name} for {label_file} does not exist. Skipping.")
                continue

            print(f"Processing: {image_name} and {label_file}")

            scaling_factor = compare_area(reference_area, current_area)
            print(f"Scaling factor for {label_file}: {scaling_factor}")

            # Process the image and label
            try:
                scale_and_crop_image(image_path, scaling_factor, label_path, json_label_path, output_folder)
                print(f"Processed and saved: {image_name} and {label_file}")
            except Exception as e:
                print(f"Error processing {image_name} and {label_file}: {e}")


path_to_reference_car_label = "../data/GP22/labels/B_Ren_12.txt" # Directory to chosen reference car
output_dir_images_flipped   # Directory to images of flipped cars
output_dir_labels_flipped   # Directory to labels of flipped cars
output_dir_images_scaled           # Directory to save images and labels of scaled cars

scale_all_images_and_labels(
    path_to_reference_car_label, 
    output_dir_images_flipped, 
    path_to_gp22_labels, 
    output_dir_labels_flipped, 
    output_dir_images_scaled
)